<a href="https://colab.research.google.com/github/trungpro5398/Cat_And_Dog_Classification/blob/main/Cat_Dog_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install torch torchvision transformers datasets transformers[torch] accel typing_extensions kaleido cohere openai tiktoken gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [5]:
pip install transformers[torch] accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 61.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from transformers import ViTFeatureExtractor, ViTForImageClassification, TrainingArguments, Trainer
from datasets import load_dataset
from datasets import DatasetDict

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load the dataset
dataset = load_dataset("Bingsu/Cat_and_Dog")

# Split the dataset into train, validation, and test sets
dataset = DatasetDict({
    'train': dataset['train'].train_test_split(test_size=0.2)['train'],
    'test': dataset['train'].train_test_split(test_size=0.2)['test']
})

# Further split the train dataset into train and validation sets
dataset['train'] = dataset['train'].train_test_split(test_size=0.25)

# At this point, dataset contains train, validation, and test sets
train_dataset = dataset['train']['train']
val_dataset = dataset['train']['test']
test_dataset = dataset['test']

# Apply transformations to datasets (example transformation)
# Replace 'transform' with your actual transformation function
train_dataset = train_dataset.map(lambda example: {'pixel_values': transform(example['image']), 'labels': example['labels']})
val_dataset = val_dataset.map(lambda example: {'pixel_values': transform(example['image']), 'labels': example['labels']})
test_dataset = test_dataset.map(lambda example: {'pixel_values': transform(example['image']), 'labels': example['labels']})




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/4800 [00:00<?, ? examples/s]

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

In [2]:
# Load the ViT model and feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k", num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)
# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=None,  # You can customize the data collator if needed
    train_dataset=train_dataset,
    eval_dataset=val_dataset  # Use val_dataset directly
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,0.695200
20,0.681600
30,0.677400
40,0.655400
50,0.632500
60,0.600700
70,0.558600
80,0.509000
90,0.447000
100,0.382300


TrainOutput(global_step=450, training_loss=0.17229226590858565, metrics={'train_runtime': 1434.6385, 'train_samples_per_second': 10.037, 'train_steps_per_second': 0.314, 'total_flos': 1.1158846504501248e+18, 'train_loss': 0.17229226590858565, 'epoch': 3.0})

In [3]:
eval_results = trainer.evaluate()
print(eval_results)


{'eval_loss': 0.045675136148929596, 'eval_runtime': 152.659, 'eval_samples_per_second': 10.481, 'eval_steps_per_second': 0.328, 'epoch': 3.0}


In [12]:
# Define the directory where you want to save the model
model_save_directory = "/content/my_model"

# Save the model and the tokenizer/feature extractor
model.save_pretrained(model_save_directory)
feature_extractor.save_pretrained(model_save_directory)

['/content/my_model/preprocessor_config.json']

In [13]:
import gradio as gr

model_save_directory = "/content/my_model"

# Function to load the model and feature extractor
def load_model_and_feature_extractor():
    model = ViTForImageClassification.from_pretrained(model_save_directory)
    feature_extractor = ViTFeatureExtractor.from_pretrained(model_save_directory)
    return model, feature_extractor

# Load the model and feature extractor
model, feature_extractor = load_model_and_feature_extractor()
# This is your prediction function that Gradio will use
def predict_image_with_gradio(image):
    # Transform the image to the format expected by the model
    image = feature_extractor(images=image, return_tensors="pt").to(model.device)

    # Get predictions
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        outputs = model(**image)

    # Convert output logits to softmax probabilities
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)

    # Get the top prediction
    top_prob, top_catid = torch.topk(probabilities, 1)
    predicted_label = model.config.id2label[top_catid.item()]
    probability = top_prob.item()

    # Construct the results dictionary with Python's if-else expressions
    return {
        "Cat": probability if predicted_label == 'LABEL_0' else (1 - probability),
        "Dog": probability if predicted_label == 'LABEL_1' else (1 - probability)
    }


# Define the input and output components for the Gradio interface
input_component = "image"
output_component = gr.Label(num_top_classes=2)

# Create the Gradio interface
iface = gr.Interface(
    fn=predict_image_with_gradio,
    inputs=input_component,
    outputs=output_component,
    title="Cat and Dog Image Classifier",
    description="Upload an image and the model will predict whether it's a cat or dog.",
)

iface.launch()

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://06d441377d3faf9925.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
